<div>
    <img src="images/emlyon.png" style="height:60px; float:left; padding-right:10px; margin-top:5px" />
    <span>
        <h1 style="padding-bottom:5px;"> Smart Supply Chain </h1>
        <a href="https://masters.em-lyon.com/fr/msc-in-data-science-artificial-intelligence-strategy">[Emlyon]</a> MSc in Data Science & Artificial Intelligence Strategy (DSAIS) <br/>
         February 2023, Paris | © Saeed VARASTEH
    </span>
</div>

### Part 02 : Time Series Forecasting | Statistical Models

This lecture content will be about statistical models in time series forecasting.

---

### Import data and modules

#### Import modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore")

#### Import data

In [ ]:
df = pd.read_csv('./data/international-airline-passengers.csv',header=None)
df.columns = ['year','passengers']
print(df.shape)
df.head(5)

#### Convert year column to datetime

In [ ]:
df['year'] = pd.to_datetime(df['year'], format='%Y-%m')

In [ ]:
df.head(5)

#### Set year as index

In [ ]:
df.set_index('year', inplace=True, drop=True)
df.head(5)

#### Set monthly frequency for `df`

In [ ]:
df = df.asfreq('MS')
df.head()

---

### Prepare our time series

#### Making time series stationary

In [ ]:
df_log = np.log(df)
dfs = df_log - df_log.shift(1) 
dfs.dropna(inplace=True)

df_old = df.copy()
df = dfs
df.head()

#### Autocorrelation and Partial autocorrelation plots

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

plot_acf(df, lags = 15)
plot_pacf(df, lags = 15)
plt.show()

---

### Time series forecasting

There are many methods that we can use for time series forecasting and there is not a clear winner. Model selection should always depend on how you data look and what are you trying to achieve.

#### Split our dataset to be able to evaluate our models

In [ ]:
split_date ='1959-01-01'
df_train = df.loc[df.index < split_date]
df_test = df.loc[df.index >= split_date]
print(f"{len(df_train)} months of training data and {len(df_test)} months of testing data ")

In [ ]:
df_test.head()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15, 5))
plt.plot(df_train, label="train");
plt.plot(df_test, label="test", color="green");
plt.legend();

### <span style="color:steelblue;"> Naive forecast </span>

In [ ]:
df_mean = df_train.passengers.mean()
mean_forecast = np.array([df_mean for v in range(len(df_test))])
mean_forecast_df = pd.DataFrame(data=mean_forecast, columns=df_test.columns, index=df_test.index)

In [ ]:
def plot_forecast(forecast_df,method_name):
    fig, ax = plt.subplots(1,1,figsize=(15, 5))
    plt.plot(df_train, label="train");
    plt.plot(df_test, label="test", color="green");
    plt.plot(forecast_df, label=method_name, color="red", linestyle=":");
    plt.legend();

In [ ]:
plot_forecast(mean_forecast_df, "Naive")

---

### Forecast quality scoring metrics

__Mean Squared Error (MSE)__, most commonly used, gives higher penalty to big mistakes and vise versa, [0, +inf).

__Mean Absolute Error (MAE)__, it is an interpretable metric because it has the same unit of measurement as the initial series, [0, +inf).

__Mean Absolute Percentage Error (MAPE)__, same as MAE but percentage, — very convenient when you want to explain the quality of the model to your management, [0, +inf).

__R squared ($R^2$)__, coefficient of determination (it can be interpreted as a percentage of variance explained by the model), (-inf, 1].

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

In [ ]:
results_all_models = pd.DataFrame(columns=["MSE","MAE","MAPE","R2"])

def evaluate_forecast(df_test, df_pred, method_name):
    y = df_test.values
    y_pred = df_pred.values
    results = {}
    results['MSE'] = mean_squared_error(y, y_pred)
    results['MAE'] = mean_absolute_error(y, y_pred)
    results['MAPE'] = mean_absolute_percentage_error(y, y_pred)
    results['R2'] = r2_score(y, y_pred)
    
    results_df = pd.DataFrame(data=results, index=[method_name])
    global results_all_models
    results_all_models = results_all_models.append(results_df)
    print( results_all_models.head(10) )

In [ ]:
evaluate_forecast(df_test, mean_forecast_df, "Naive")

---

### <span style="color:steelblue;"> Autoregression (AR) </span>

The autoregression (AR) method models the next step in the sequence as a linear function of the observations at prior time steps. 

Parameters of the model:

- __Number of AR (Auto-Regressive) terms (p):__ p is the parameter associated with the auto-regressive aspect of the model, which incorporates past values i.e lags of dependent variable. For instance if p is 5, the predictors for x(t) will be based on x(t-1)….x(t-5).


In [ ]:
from statsmodels.tsa.arima_model import ARMA

pred_ = list()
for t in range(len(df_test.passengers)):
    temp_train = df[:len(df_train)+t]
    model = ARMA(temp_train.passengers, order=(1, 0)) # (AR order, MA order)
    model_fit = model.fit(disp=False)
    predictions = model_fit.predict(start=len(temp_train), end=len(temp_train), dynamic=False)
    pred_ = pred_ + [predictions]
    
ar_forecast_df = pd.concat(pred_)

In [ ]:
plot_forecast(ar_forecast_df, "AR forecast")

In [ ]:
evaluate_forecast(df_test, ar_forecast_df, "AR")

### <span style="color:steelblue;"> Moving Average (MA) </span>

The Moving Average (MA) method models the next step in the sequence as the average of a window of observations at prior time steps. 

Parameters of the model:

- __Number of MA (Moving Average) terms (q):__ q is size of the moving average part window of the model i.e. lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be based on e(t-1)….e(t-5) where e(i) is the difference between the moving average at ith instant and actual value. 


In [ ]:
from statsmodels.tsa.arima_model import ARMA

pred_ = list()
for t in range(len(df_test.passengers)):
    temp_train = df[:len(df_train)+t]
    model = ARMA(temp_train.passengers, order=(0, 1))
    model_fit = model.fit(disp=False)
    predictions = model_fit.predict(start=len(temp_train), end=len(temp_train), dynamic=False)
    pred_ = pred_ + [predictions]
    
ma_forecast_df = pd.concat(pred_)

In [ ]:
plot_forecast(ma_forecast_df, "MA forecast")

In [ ]:
evaluate_forecast(df_test, ma_forecast_df, "MA")

### <span style="color:steelblue;"> Autoregressive Moving Average (ARMA) </span>

This method will basically join the previous two `AR` and `MA`. Model parameters will be the sum of the two.

Parameters of the model:

- __Number of AR (Auto-Regressive) terms (p)__ 
- __Number of MA (Moving Average) terms (q)__ 

In [ ]:
from statsmodels.tsa.arima_model import ARMA

pred_ = list()
for t in range(len(df_test.passengers)):
    temp_train = df[:len(df_train)+t]
    model = ARMA(temp_train.passengers, order=(1, 1))
    model_fit = model.fit(disp=False)
    predictions = model_fit.predict(start=len(temp_train), end=len(temp_train), dynamic=False)
    pred_ = pred_ + [predictions]
    
arma_forecast_df = pd.concat(pred_)

In [ ]:
plot_forecast(arma_forecast_df, "ARMA forecast")

In [ ]:
evaluate_forecast(df_test, arma_forecast_df, "ARMA")

### <span style="color:steelblue;">  Autoregressive integrated Moving Average (ARIMA)  </span>

In an ARIMA model there are 3 parameters that are used to help model the major aspects of a times series: seasonality, trend, and noise. 

These parameters are labeled p,d,and q:

* __Number of AR (Auto-Regressive) terms (p)__

* __Number of Differences (d):__ d is the parameter associated with the integrated part of the model, which effects the amount of differencing to apply to a time series.

* __Number of MA (Moving Average) terms (q)__

<div class="alert-warning">
Note: A problem with ARIMA is that it does not support seasonal data. That is a time series with a repeating cycle. ARIMA expects data that is either not seasonal or has the seasonal component removed, e.g. seasonally adjusted via methods such as seasonal differencing.
</div>

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

pred_ = list()
for t in range(len(df_test.passengers)):
    temp_train = df[:len(df_train)+t]
    model = ARIMA(temp_train.passengers, order=(1, 0, 1))
    model_fit = model.fit(disp=False)
    predictions = model_fit.predict(start=len(temp_train), end=len(temp_train), dynamic=False)
    pred_ = pred_ + [predictions]
    
arima_forecast_df = pd.concat(pred_)

In [ ]:
plot_forecast(arima_forecast_df, "ARIMA forecast")

In [ ]:
evaluate_forecast(df_test, arima_forecast_df, "ARIMA")

### <span style="color:steelblue;">  Seasonal Autoregressive Integrated Moving-Average (SARIMA)  </span>

Seasonal Autoregressive Integrated Moving Average, SARIMA or Seasonal ARIMA, is an extension of ARIMA that explicitly supports univariate time series data with a seasonal component.

It adds three new hyperparameters to specify the autoregression (AR), differencing (I) and moving average (MA) for the seasonal component of the series, as well as an additional parameter for the period of the seasonality.

__Trend Elements:__

There are three trend elements that require configuration. They are the same as the ARIMA model, specifically:

- p: Trend autoregression order.
- d: Trend difference order.
- q: Trend moving average order.

__Seasonal Elements:__

There are four seasonal elements that are not part of ARIMA that must be configured; they are:

- P: Seasonal autoregressive order.
- D: Seasonal difference order.
- Q: Seasonal moving average order.
- m: The number of time steps for a single seasonal period. For example, an S of 12 for monthly data suggests a yearly seasonal cycle.

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

pred_ = list()
for t in range(len(df_test.passengers)):
    temp_train = df[:len(df_train)+t]
    model = SARIMAX(temp_train.passengers, order=(1, 0, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    predictions = model_fit.predict(start=len(temp_train), end=len(temp_train), dynamic=False)
    pred_ = pred_ + [predictions]
    
sarima_forecast_df = pd.concat(pred_)

In [ ]:
plot_forecast(sarima_forecast_df, "SARIMA forecast")

In [ ]:
evaluate_forecast(df_test, sarima_forecast_df, "SARIMA")

---

<div class="alert-danger">
To test the Exponential Smoothing methods, we come back to the original series.
</div>

In [ ]:
df = df_old
df_train = df.loc[df.index < split_date]
df_test = df.loc[df.index >= split_date]
print(f"{len(df_train)} months of training data and {len(df_test)} months of testing data ")

---

### <span style="color:steelblue;">  Simple Exponential Smoothing (SES) </span>

The Simple Exponential Smoothing (SES) method models the next time step as an exponentially weighted linear function of observations at prior time steps. This method expects our time series to be stationary in order to perform adecuately (no trend or seasonality).

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

pred_ = list()
for t in range(len(df_test.passengers)):
    temp_train = df[:len(df_train)+t]
    model = SimpleExpSmoothing(temp_train.passengers)
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
    pred_ = pred_ + [predictions]
    
ses_forecast_df = pd.concat(pred_)

In [ ]:
plot_forecast(ses_forecast_df, "SES forecast")

In [ ]:
evaluate_forecast(df_test, ses_forecast_df, "SES")

### <span style="color:steelblue;">  Holt Winter’s Exponential Smoothing (HWES) </span>

HWES also known as triple exponential smoothing.

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

pred_ = list()
for t in range(len(df_test.passengers)):
    temp_train = df[:len(df_train)+t]
    model = ExponentialSmoothing(temp_train.passengers, seasonal_periods = 12, trend='additive', seasonal='additive')
    model_fit = model.fit(optimized=True)
    predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
    pred_ = pred_ + [predictions]
    
hwes_forecast_df = pd.concat(pred_)

In [ ]:
plot_forecast(hwes_forecast_df, "HWES forecast")

In [ ]:
evaluate_forecast(df_test, hwes_forecast_df, "HWES")

---

<div class="alert-info" style="background-color:#ece4f5;">
    <span style="font-weight:bold; color:#8966b0;">
        <h4 style="padding-top:4px; padding-bottom:4px"> EXERCISES 01 - TRY IT YOURSELF </h4>
    </span>
</div>

---